In [1]:
import praw
import re
import json

# Set up Reddit API client
reddit = praw.Reddit(
    client_id="Efbot9iva7bBckLwx-Ic-g",
    client_secret="OWxn-YzLwhj0nDMPb7Ai72gQxVJjmg",
    user_agent="prescribed-burning",  # e.g., "prescribed_burn_study"
)

# List of subreddits you want to search in
subreddits = ["Oregon", "Portland", "California", "LosAngeles", "PNW", "Colorado", "rural",
    "Alaska", "Montana", "firefighting", "forestry", "wildfires"]

# List of search terms related to prescribed burns
search_terms = [
    "prescribed burns", "controlled burns", "fire management", "wildfire prevention", 
    "forest management", "fire ecology", "land management", "fuel reduction", 
    "burning policies", "wildfire policy", "firebreaks"
]

# Combine search terms into a single query
search_query = ' OR '.join(search_terms)

# Initialize the results list
results = []

# Function to check for location keywords in text
def extract_location(text):
    location_keywords = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]
    for location in location_keywords:
        if location in text:
            return location
    return None

# Search posts in each subreddit
for subreddit in subreddits:
    print(f"Searching in r/{subreddit}...")
    for post in reddit.subreddit(subreddit).search(search_query, limit=50):  # Limit to 50 posts
        post_data = {
            "title": post.title,
            "score": post.score,
            "url": post.url,
            "num_comments": post.num_comments,
            "comments": [],
            "replies": [],
            "location": extract_location(post.title) or extract_location(post.selftext)
        }
        
        # Fetch comments (replace "more comments" and limit to 0 to get all comments)
        post.comments.replace_more(limit=0)

        # Add comments and replies to the post data
        count = 0
        
        for comment in post.comments.list():  # All comments
            if count >= 1:
                post_data["replies"].append(comment.body)
            else:
                post_data["comments"].append(comment.body)
            count += 1

        # Add post data to the results list
        results.append(post_data)

# Save results to a file
import json
with open("reddit_prescribed_burns.json", "w") as f:
    json.dump(results, f, indent=4)

print("Data saved to reddit_prescribed_burns.json")

Searching in r/Oregon...
Searching in r/Portland...
Searching in r/California...
Searching in r/LosAngeles...
Searching in r/PNW...
Searching in r/Colorado...
Searching in r/rural...
Searching in r/Alaska...
Searching in r/Montana...
Searching in r/firefighting...
Searching in r/forestry...
Searching in r/wildfires...
Data saved to reddit_prescribed_burns.json
